In [37]:
import pandas

In [2]:
%load_ext autoreload
%autoreload 2
import os

os.chdir('../')

from data_scripts.data_schema import DataSchema

os.chdir('./notebooks')

In [3]:
data = pandas.read_csv("../../data/raw/2014.csv", dtype=DataSchema.schema, parse_dates=DataSchema.dates)

In [8]:
data

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1361929741,FCJ5493,NY,PAS,12/18/1970,20,SUBN,GMC,S,35030,...,BLACK,0,2013,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
1,1366962000,63540MC,NY,COM,02/02/1971,46,DELV,FRUEH,P,58830,...,BRN,0,2013,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
2,1342296187,GCY4187,NY,SRF,09/18/1971,21,VAN,FORD,S,11790,...,BLUE,0,2002,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
3,1342296199,95V6675,TX,PAS,09/18/1971,21,<NA>,GMC,S,11790,...,SILVR,0,2008,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
4,1342296217,FYM5117,NY,SRF,09/18/1971,21,SUBN,NISSA,S,28190,...,WHITE,0,2012,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9100270,1369104972,40822ME,NY,COM,06/26/2066,46,VAN,FORD,P,11710,...,WHITE,0,2013,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
9100271,1361802455,CDE2330,NY,PAS,11/03/2066,46,SUBN,CHEVR,P,59430,...,GY,0,2004,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
9100272,1365797922,CRN7564,NY,PAS,01/30/2067,40,VAN,FORD,P,73480,...,WHITE,0,2003,-,0,<NA>,<NA>,<NA>,<NA>,<NA>
9100273,1367760884,586ZGA,CT,PAS,04/15/2067,40,SUBN,CHEVR,P,77930,...,TAN,0,0,-,0,<NA>,<NA>,<NA>,<NA>,<NA>


In [46]:
data_processed = DataSchema.to_primitive_dtypes(DataSchema.fill_na(data))

# Saving Parquet File

In [6]:
data_processed.to_parquet("../../data/parquet/2014.parquet")

In [45]:
# READING PARQUET
# parquet_data = pandas.read_parquet("../../data/parquet/2014.parquet")

# Saving HDF5 File

In [8]:
import numpy as np

def process_type(type):
    if type == np.int64:
        return "int64"
    if type == np.dtype("O"):
        return "S1"
    return "datetime64[D]"

data_types = [(name, process_type(type)) for name, type in data_processed.dtypes.items()]

In [9]:
array = np.empty(len(data_processed), dtype=data_types)

for column in data_processed.columns:
    array[column] = data_processed[column]

In [10]:
import h5py

with h5py.File('../../data/hdf5/2014.h5','w') as h5f:
    h5f.create_dataset('data', data=array, compression="gzip", compression_opts=9)

In [44]:
# READING HDF5
# data = []
# with h5py.File('../../data/hdf5/2014.h5', "r") as f:
#     data = f["data"][()]

# pandas.DataFrame(data, columns=DataSchema.schema.keys())